In [ ]:
import os
import tarfile
from dotenv import load_dotenv
from landsatxplore.api import API
from landsatxplore.earthexplorer import EarthExplorer

In [ ]:
# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

# Récupérer les identifiants USGS depuis les variables d'environnement
username = os.getenv('USGS_USERNAME')
password = os.getenv('USGS_PASSWORD')

# Initialiser l'API LandsatXplore
api = API(username, password)

In [ ]:
# Rechercher des images Landsat TM (B3 et B4) sur le site de Simandou
scenes = api.search(
    dataset='LANDSAT_TM_C2_L1',
    latitude=9.16,
    longitude=-9.16,
    start_date='1995-07-01',
    end_date='2020-09-01',
    max_cloud_cover=10
)

print(f"{len(scenes)} scènes trouvées.")

In [ ]:
# Créer le répertoire de sortie pour les téléchargements
output_dir = '../data/raw/landsat'
os.makedirs(output_dir, exist_ok=True)

# Initialiser EarthExplorer pour le téléchargement
ee = EarthExplorer(username, password)

In [ ]:
# Fonction pour extraire uniquement les bandes B3 et B4
def extract_b3_b4(tar_path, dest_dir):
    """ Extraction des bandes B3 (Rouge) et B4 (NIR) depuis l'archive TAR """
    with tarfile.open(tar_path, 'r') as tar:
        members = tar.getmembers()
        # Filtrer pour ne garder que les fichiers des bandes B3 et B4
        b3_b4_members = [m for m in members if '_B3.TIF' in m.name or '_B4.TIF' in m.name]
        
        # Si on trouve les bandes B3 et B4, on les extrait
        if b3_b4_members:
            tar.extractall(path=dest_dir, members=b3_b4_members)
            print(f"Bandes B3 et B4 extraites dans : {dest_dir}")
        else:
            print(f"Pas de bandes B3 ou B4 trouvées dans : {tar_path}")

In [ ]:
# Télécharger et organiser les images par date
for scene in scenes:
    acquisition_date = scene['acquisition_date'].strftime('%Y-%m-%d')
    product_id = scene['landsat_product_id']
    print(f"\nTéléchargement de la scène : {product_id} pour la date : {acquisition_date}")
    
    # Créer un dossier par date
    date_dir = os.path.join(output_dir, acquisition_date)
    os.makedirs(date_dir, exist_ok=True)
    
    # Chemin de l'archive TAR à télécharger
    tar_path = os.path.join(date_dir, f"{product_id}.tar")
    
    # Télécharger l'image si elle n'existe pas déjà
    if not os.path.exists(tar_path):
        try:
            ee.download(product_id, output_dir=date_dir)
            print(f"Téléchargement terminé : {tar_path}")
            
            # Extraire uniquement les bandes B3 et B4
            extract_b3_b4(tar_path, date_dir)
            
            # Supprimer l'archive TAR après extraction
            os.remove(tar_path)
            print(f"Archive TAR supprimée : {tar_path}")
            
        except Exception as e:
            print(f"Erreur lors du téléchargement de la scène {product_id}: {e}")
    else:
        print(f"La scène {product_id} a déjà été téléchargée.")
        
# Déconnexion de l'API et de EarthExplorer
api.logout()
ee.logout()

print("\nTéléchargements terminés et images organisées par date.")